<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deep-daze stomp-py --upgrade

     |████████████████████████████████| 1.4MB 12.8MB/s 
     |████████████████████████████████| 3.3MB 71.9MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 92kB 12.0MB/s 
  Created wheel for ftfy: filename=ftfy-6.0-cp37-none-any.whl size=41622 sha256=3614181ebd178656d5a08044faabb43c6fa9ebe7133ed1c00117a995ed4e10d6
  Stored in directory: /root/.cache/pip/wheels/22/8b/08/7d1c17849e10371206a262304973b5a9f45e8b9d0a2179f465
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=c7e8d6da9901199d1cf221a88237bf62eb873a39505501603d499b5e81902e46
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built ftfy fire
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling ima

In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
flatten = itertools.chain.from_iterable

WATCH_MEDIA_FILES = ["./*.jpg","./*.mp4"]

conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)


!rm *.jpg

p = Popen('imagine --iterations=50 --epochs=5 --save-every=5 --save-progress=True --open_folder=False --overwrite=True "A slick high-contrast"', stdout = PIPE, 
        stderr = STDOUT, shell = True)

FOUND_MEDIA_FILES=[]
while True:
  line = p.stdout.readline()#
  if not line: break

  stdout.write(line)
  conn.send(body=line.decode("utf-8"), destination='/topic/colabOut')
  NEW_FOUND_MEDIA_FILES=list(flatten([glob(search_path) for search_path in WATCH_MEDIA_FILES]))
  NEW_FOUND_MEDIA_FILES.sort(key=os.path.getmtime)
  ADDED_MEDIA_FILES = list(set(NEW_FOUND_MEDIA_FILES)-set(FOUND_MEDIA_FILES))
  FOUND_MEDIA_FILES = NEW_FOUND_MEDIA_FILES
  conn.send(body=json.dumps(ADDED_MEDIA_FILES), destination='/topic/colabMediaOut')
conn.disconnect()

epochs:   0%|          | 0/5 [00:00<?, ?it/s]
                                             
loss: -23.28:   8%|▊         | 4/50 [00:01<00:17,  2.58it/s]
                                             
loss: -25.04:  18%|█▊        | 9/50 [00:03<00:15,  2.62it/s]
                                             
loss: -26.36:  28%|██▊       | 14/50 [00:05<00:13,  2.62it/s]
                                             
loss: -26.89:  38%|███▊      | 19/50 [00:07<00:11,  2.63it/s]
                                             
loss: -27.23:  42%|████▏     | 21/50 [00:08<00:11,  2.57it/s]


In [ ]:
from glob import glob
import requests
import requests
from tqdm import tqdm
!mkdir -p results
for imgfile in tqdm(glob("./A_slick*.jpg")):
  print("Super resolution of", imgfile)
  
  r = requests.post(
    "https://api.deepai.org/api/waifu2x",
    files={
        'image': open(imgfile, 'rb'),
    },
    headers={'api-key': "blalasd"}
  )
  srurl = r.json()["output_url"]
  with open(f"/content/results/{imgfile}.upsampled.jpg", 'wb') as f:
    f.write(requests.get(srurl).content)
  #!echo curl -F f"\'image=@{imgfile}\' -H api-key: https://api.deepai.org/api/torch-srgan" 

In [ ]:
!mkdir -p images
!ffmpeg -r 20 -y -i results/A_slick%*.jpg -filter:v scale=512:-1 images/env_school_flashy.mp4

In [ ]:
!pip install stomp-py